In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import root_pandas as rp
import pandas as pd
from tqdm import tqdm
from scipy.special import erfc

from scipy.optimize import curve_fit

In [ ]:
def model(x, a, b, c):
    """
    a => sigma_l + sigma_t
    b => epsilon*sigma_tt
    c => Sqrt(2epsilon(1+epsilon))* sigma_lt
    """
    f = a + b * np.cos(2*x) + c * np.cos(x)
    return f

def degauss(x, A, mu, sigma, lambda1, lambda2):
    mu1 = sigma * sigma * lambda1 + x - mu
    mu2 = -sigma * sigma * lambda2 + x - mu
    ret = A * 0.5 / (1.0 / lambda1 + 1.0 / lambda2) * \
        (np.exp(0.5 * np.power(sigma * lambda1, 2) + lambda1 * (x - mu)) * erfc(mu1 / (sigma * np.sqrt(2.0))) \
         + np.exp(0.5 * np.power(sigma * lambda2, 2) - lambda2 * (x - mu)) * erfc(-mu2 / (sigma * np.sqrt(2.0))))

    return ret

def gauss(x, A, mu, sig):
    ret = np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
    return A*ret

def peak(x, c):
    return np.exp(-np.power(x - c, 2) / 16.0)

def lin_interp(x, y, i, half):
    return x[i] + (x[i+1] - x[i]) * ((half - y[i]) / (y[i+1] - y[i]))

def half_max_x(x, y):
    half = np.max(y)/2.0
    signs = np.sign(np.add(y, -half))
    zero_crossings = (signs[0:-2] != signs[1:-1])
    zero_crossings_i = np.where(zero_crossings)[0]
    return [lin_interp(x, y, zero_crossings_i[0], half),
            lin_interp(x, y, zero_crossings_i[1], half)]


In [ ]:
#df = pd.read_csv("test.csv",index_col=False)
#df = pd.read_csv("all.csv", index_col=False)
df = pd.read_csv("/Users/tylern/physics_code/current/build/e1d_mc.csv", index_col=False)
    
df = df[(df.w > 0) & (df.mm2 > 0.5) & (df.mm2 < 1.5)]
df['cos_theta'] = np.cos(df.theta)
#df = df[np.abs(df.helicty) == 1]
print(len(df))
df.head()

In [ ]:
def mm_cut(df):
    NSIGMA = 3
    data = []
    for sec in range(1,7):
        plt.figure(figsize=(12,9))
        y,x = np.histogram(df[df.electron_sector == sec].mm2, bins=500, density=True)
        x=(x[1:]+x[:-1])/2
        popt_g, pcov_g = curve_fit(gauss,x,y, maxfev=8000)
        plt.plot(x, gauss(x,*popt_g), linewidth=2.0)
        plt.errorbar(x,y,yerr=np.sqrt(y.shape[0])/y.shape[0], fmt='.', zorder=1)

        plt.axvline(popt_g[1]+NSIGMA*popt_g[2])
        plt.axvline(popt_g[1]-NSIGMA*popt_g[2])
        
#         p0 = [popt_g[0],popt_g[1],popt_g[2], 1.0,1.0]
#         popt, pcov = curve_fit(degauss,x,y, maxfev=8000)
        
#         plt.plot(x, degauss(x,*popt), c='#9467bd', linewidth=2.0)
        
#         # find the FWHM
#         xs= np.linspace(0.7,1.5,100000)
#         hmx = half_max_x(xs, degauss(xs,*popt))
#         fwhm = hmx[1] - hmx[0]
#         plt.axvline(popt[1]+NSIGMA*fwhm/2.355, c='#9467bd')
#         plt.axvline(popt[1]-NSIGMA*fwhm/2.355, c='#9467bd')
        
        plt.show()

        data.append(df[(df.electron_sector == sec)
                       &(df.mm2 <= popt_g[1]-NSIGMA*popt_g[2])
                       &(df.mm2 >= popt_g[1]+NSIGMA*popt_g[2])])
        
        print('{', end='')
        for x in popt_g:
            print(f' {x:.20f},', end='')
        print('}')
        print(data)
    return data

In [ ]:
data = mm_cut(df)
all_data = pd.concat(data)

In [ ]:
plt.figure(figsize=(12,9))
for i in range(0,6):
    #y, x = np.histogram(data[i].mm2, bins=200, density=False, range=(0.2, 1.0))
    y, x = np.histogram(data[i].mm2, bins=200, density=True, range=(0.7, 1.05))
    x = (x[1:]+x[:-1])/2
    plt.hist(data[i].mm2, bins=200, density=True, range=(0.7, 1.05))
    #plt.scatter(x[y!=0], y[y!=0], label=f'sector {i+1}')
plt.legend()
plt.show()

In [ ]:
all_data.head()

In [ ]:
plt.figure(figsize=(12,9))
plt.hist2d(all_data.cos_theta, all_data.phi, bins=(15,10))
plt.show()

In [ ]:
NUM_BINS_W = 12
NUM_BINS_Q2 = 4
NUM_BINS_PHI = 12
NUM_BINS_CT = 10

W_MIN = 1.2
W_MAX = 1.8
Q2_MIN = 1.5
Q2_MAX = 3.5



plt.figure(figsize=(16,10))
counts, w_bins, q2_bins, mesh = plt.hist2d(all_data.w,all_data.q2, 
                                           bins=(NUM_BINS_W, NUM_BINS_Q2), 
                                           range=((W_MIN, W_MAX),(Q2_MIN, Q2_MAX)), 
                                           density=True)
plt.colorbar()
plt.show()

plt.figure(figsize=(16,10))
counts, phi_bins, theta_bins, mesh = plt.hist2d(all_data.phi, all_data.cos_theta, 
                                                bins=(NUM_BINS_PHI, NUM_BINS_CT), 
                                                density=True)
plt.colorbar()
plt.show()

In [ ]:
cross_sections = df[["w","q2","cos_theta","phi","helicty"]].copy(deep=True)

In [ ]:
cross_sections = df[["w","q2","cos_theta","phi","helicty"]].copy(deep=True)
w_bins = np.arange(1.2, 1.8, 0.025)
q2_bins = np.arange(1.0, 3.5, 0.5)
theta_bins = np.arange(-1.0, 1.0, 0.25)


cross_sections['w_bin'] = pd.cut(cross_sections['w'], bins=w_bins, include_lowest=True)
cross_sections['q2_bin'] = pd.cut(cross_sections['q2'], bins=q2_bins, include_lowest=True)
cross_sections['theta_bin'] = pd.cut(cross_sections['cos_theta'], bins=theta_bins, include_lowest=True)



cross_sections.dropna(inplace=True)

cross_sections.head()

In [ ]:
plt.hist2d(cross_sections.w,cross_sections.q2,bins=(len(w_bins),len(q2_bins)))
plt.show()

In [ ]:
pos = cross_sections[cross_sections.helicty == 1]
print(len(pos))
neg = cross_sections[cross_sections.helicty == -1]
print(len(neg))
noh = cross_sections[cross_sections.helicty == 0]

print(len(noh))

In [ ]:
plt.hist2d(pos.w,pos.q2,bins=(len(w_bins),len(q2_bins)))
plt.show()

plt.hist2d(neg.w,neg.q2,bins=(len(w_bins),len(q2_bins)))
plt.show()

plt.hist2d(noh.w,noh.q2,bins=(len(w_bins),len(q2_bins)))
plt.show()

In [ ]:
def draw_xsection(cross_sections, func):
    for w in np.unique(cross_sections.w_bin)[:2]:
        for q2 in np.unique(cross_sections.q2_bin):
            for cos_t in np.unique(cross_sections.theta_bin):
                fig = plt.figure(figsize=(12,9))
                data = cross_sections[(w == cross_sections.w_bin) 
                              & (q2 == cross_sections.q2_bin)
                              & (cos_t == cross_sections.theta_bin)]
                y, x = np.histogram(data.phi, bins=10, range=(0, 2*np.pi))
                x = (x[1:]+x[:-1])/2.0

                popt, pcov = curve_fit(func, x, y, maxfev=8000)
                plt.errorbar(x, y, yerr=np.sqrt(y)/len(y), marker='*', linestyle='', zorder=1)
                xs = np.linspace(0,2*np.pi,100)
                plt.plot(xs, func(xs, *popt), c='#9467bd', linewidth=2.0)
                plt.title(f"W={w}, $Q^2$={q2}, cos($\Theta$)={cos_t}")
                #plt.ylim(-5, 100)
                plt.show()
                #plt.close()


In [ ]:
draw_xsection(cross_sections, model)